In [1]:
# coding: utf-8
import requests,re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def importSoup(NomEnt):
    url = r"https://www.reuters.com/finance/stocks/financial-highlights/" + NomEnt
    res = requests.get(url)
    if res.status_code == 200:
        html_doc =  res.text
        soup = BeautifulSoup(html_doc,"html.parser")
    return soup

In [16]:
def rechercheChamps(soup,dicValeurs):

    text_Targeted = "Quarter Ending Dec-18"
    dicValeurs[text_Targeted] = soup.find_all("tr",attrs={'class':'stripe'})[0].contents[5].text
    
    text_Targeted = "Prix Action"      
    dicValeurs[text_Targeted] = soup.find_all('span', attrs={'style':'font-size: 23px;'})[0].string.strip()
    
    text_Targeted = "valueContentPercent"
    dicValeurs[text_Targeted] = soup.find_all("span",attrs={'class':text_Targeted})[0].text.strip()[1:-1]

    text_Targeted = "% Shares Owned:"
    dicValeurs[text_Targeted] = soup.find("td",text = text_Targeted).findNext("td").text
                
    text_Targeted = "Dividend Yield"
    dicValeurs[text_Targeted + " company"] = soup.find("td",text = text_Targeted).findNext("td").text
    dicValeurs[text_Targeted + " industry"] = soup.find("td",text = text_Targeted).findNext("td").findNext("td").text
    dicValeurs[text_Targeted + " sector"] = soup.find("td",text = text_Targeted).findNext("td").findNext("td").findNext("td").text
    
    return dicValeurs

In [17]:
def rechercheUrl(query):
    url = "https://www.reuters.com/finance/stocks/lookup?search=" + query + "&searchType=any&sortBy=&dateRange=&comSortBy=marketcap"
    res = requests.get(url)
    if res.status_code == 200:
        html_doc =  res.text
        soup = BeautifulSoup(html_doc,"html.parser")
        lurl = soup.find_all("tr",attrs={'class':'stripe'})
        for ur in lurl:
            if re.search(".PA",ur.contents[3].text):
                return ur.contents[3].text

In [18]:
dico={}
dicValeurs = {}
listEnt = ("airbus","LVMH","Danone")

for query in listEnt:
    del dicValeurs
    dicValeurs = {}
    NomEnt = rechercheUrl(query)
    soup = importSoup(NomEnt)
    rechercheChamps(soup,dicValeurs)
    dico[query]=dicValeurs
    
print(dico)

{'airbus': {'Quarter Ending Dec-18': '23,493.00', 'Prix Action': '99.01', 'valueContentPercent': '+1.73%', '% Shares Owned:': '43.53%', 'Dividend Yield company': '1.45', 'Dividend Yield industry': '1.34', 'Dividend Yield sector': '1.64'}, 'LVMH': {'Quarter Ending Dec-18': '13,667.70', 'Prix Action': '264.40', 'valueContentPercent': '+0.94%', '% Shares Owned:': '20.57%', 'Dividend Yield company': '1.92', 'Dividend Yield industry': '1.70', 'Dividend Yield sector': '2.60'}, 'Danone': {'Quarter Ending Dec-18': '6,072.60', 'Prix Action': '64.11', 'valueContentPercent': '+0.28%', '% Shares Owned:': '50.61%', 'Dividend Yield company': '2.49', 'Dividend Yield industry': '2.35', 'Dividend Yield sector': '2.23'}}
